Parallel Tool Calls

LangGraph can branch into multiple paths, execute them simultaneously, and rejoin the results into one coherent response.

In [ ]:
from langgraph.graph import MessagesState
from langchain_openai import ChatOpenAI

In [ ]:
#state
class MyMessagesState(MessagesState):
    pass

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
def tool_calling_llm(state: MyMessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

AIMessage is returned from a chat model as a response to a prompt. 

This message represents the output of the model and consists of both the raw output as returned by the model together standardized fields (e.g., tool calls, usage metadata) added by the LangChain framework.

In [ ]:
from langchain_core.messages import AIMessage

In [ ]:
def weather_paris(state: MyMessagesState):
    text = get_weather("Paris")
    return {"messages": [AIMessage(content=text)]}

def weather_london(state: MyMessagesState):
    text = get_weather("London")
    return {"messages": [AIMessage(content=text)]}

If both are mentioned, it returns both. And if neither city is present, we fall back to Paris

In [ ]:
def parallel_weather_condition(state: MyMessagesState):
    last_human = ""
    for msg in reversed(state["messages"]):
        if getattr(msg, "type", "") == "human":
            last_human = (msg.content or "").lower()
            break

    targets = []
    if "paris" in last_human:
        targets.append("weather_paris")
    if "london" in last_human:
        targets.append("weather_london")

    if not targets:
        targets = ["weather_paris"]

    return targets

Join Node

In [ ]:
def combine_weather(state: MyMessagesState):
    lines = []
    for m in state["messages"]:
        if isinstance(m, AIMessage) and isinstance(m.content, str) and "The weather in" in m.content:
            lines.append(m.content)

    combined = " | ".join(lines) if lines else "No weather data found."
    return {"messages": [AIMessage(content=f"Combined: {combined}")]}


Graph Workflow

In [ ]:
from langgraph.graph import StateGraph, START, END

In [ ]:
builder = StateGraph(MyMessagesState)

In [ ]:
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("weather_paris", weather_paris)
builder.add_node("weather_london", weather_london)
builder.add_node("combine_weather", combine_weather)

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", parallel_weather_condition)
builder.add_edge("weather_paris", "combine_weather")
builder.add_edge("weather_london", "combine_weather")
builder.add_edge("combine_weather", END)

In [ ]:
graph = builder.compile()

Inference

In [ ]:
from langchain_core.messages import HumanMessage

# Case 1: both cities mentioned
print("\n=== Case 1: both cities mentioned ===")
messages = graph.invoke({"messages": [HumanMessage(content="What's the weather in Paris and London?")]})
for m in messages["messages"]:
    m.pretty_print()

# Case 2: one city mentioned
print("\n=== Case 2: one city mentioned ===")
messages = graph.invoke({"messages": [HumanMessage(content="What's the weather in Paris?")]})
for m in messages["messages"]:
    m.pretty_print()

# Case 3: no city mentioned (triggers fallback)
print("\n=== Case 3: no city mentioned (triggers fallback) ===")
messages = graph.invoke({"messages": [HumanMessage(content="Weather please")]})
for m in messages["messages"]:
    m.pretty_print()
